In [79]:
#import packages 
import requests
from bs4 import BeautifulSoup
import time
import csv
import math
import numpy as np

with open('products.csv', mode='w',  encoding='utf8') as products:
    fieldnames = ['Category', 'Subcategory', 'Title', 'Brend', 'Country', 'Url of Product']
    writer = csv.DictWriter(products, fieldnames=fieldnames)
    writer.writeheader()
    
    tm_cnt = dict() #to save time we will save trademark and country
    
    start = time.time()
    page1 = requests.get('https://contentservice.kz/produkty-pitaniya-napitki-kz/')
    soup1 = BeautifulSoup(page1.text, 'html.parser')
    temp = soup1.find('ul', class_ = 'link-list link-list_md') #getting categories 
    cat = temp.find_all('li') 
    categories = [i.text[:i.text.find('\xa0')] for i in cat] #[:i.text.find('\xa0')] needed to format 

    cat_links = [a['href'] for a in temp.find_all('a', href=True)] #getting urls 

    
    num_cat = len(categories)
    for i in range(num_cat) : 

        URL = 'https://contentservice.kz'

        page2 = requests.get(URL + cat_links[i])
        soup2 = BeautifulSoup(page2.text, 'html.parser')
        temp2 = soup2.find('ul', class_ = 'link-list link-list_md') #getting categories 
        scat = temp2.find_all('li') 
        scategories = [scats.text[:scats.text.find('\xa0')] for scats in scat] #[:i.text.find('xa')-1] needed to format 
        
        prdnum_in_cat = [scats.text[scats.text.find('\xa0')+1:] for scats in scat]
        
        scat_links = [a['href'] for a in temp2.find_all('a', href=True)] #getting urls 
        
        num_scat = len(scategories)
        for j in range(num_scat):

            
            pagenum = math.ceil(int(prdnum_in_cat[j])/24)
            
            for k in range(pagenum):
            
                page3 = requests.get(URL + scat_links[j] + 'page' + str(k+1) + '/')
                soup3 = BeautifulSoup(page3.text, 'html.parser')

                temp3 = soup3.find('div', class_ = 'catalog__grid-view')
                product_list = temp3.find_all('a', class_ = 'link-gray')
                prod = [prods.text for prods in product_list]

                tms = [span.text for span in temp3.find_all('span', class_='text-color_fade')]

                prod_urls = [a['href'] for a in product_list]

                num_prods = len(prod)

                for itr in range(num_prods):
                    prod[itr] = prod[itr][:len(prod[itr]) - len(tms[itr])-1]
    #             print(prod)

                for tm in range(num_prods):
                    if (tms[tm] in tm_cnt.keys()) == False:
                        get_cnt = requests.get('https://contentservice.kz' + prod_urls[tm])
                        cntry_html = BeautifulSoup(get_cnt.text, 'html.parser')
                        country = cntry_html.find('span', class_='flag')
                        try:
                            tm_cnt[tms[tm]] = str(country['title'])
                        except:
                            tm_cnt[tms[tm]] = np.NaN
                for itr in range(num_prods):
                    
                    writer.writerow({'Category' : categories[i],
                                     'Subcategory' : scategories[j],
                                     'Title' : prod[itr], 
                                     'Brend' : tms[itr],
                                     'Country' : tm_cnt[tms[itr]],
                                     'Url of Product': prod_urls[itr]})
                
        
print('time scraped:', time.time()-start, 'seconds')

time scraped: 887.4235932826996 seconds


In [112]:
import pandas as pd
df = pd.read_csv('products.csv')
df

,Category,Subcategory,Title,Brend,Country,Url of Product
0,Молочные продукты и яйца,Йогурт,Йогурт 2.5% из козьего молока Zeren ст 200мл,Zeren,KZ,/product/710535169287-kz-yogurt-iz-kozego-molo...
1,Молочные продукты и яйца,Йогурт,Йогурт 4% из козьего молока Zeren п/бут 300мл,Zeren,KZ,/product/710535169089-kz-yogurt-iz-kozego-molo...
2,Молочные продукты и яйца,Йогурт,Йогурт 4% из козьего молока Клубника Zeren п/б...,Zeren,KZ,/product/710535169225-kz-yogurt-klubnichnyy-iz...
3,Молочные продукты и яйца,Йогурт,Йогурт 3% Турецкий Kokshe ст 360г,Kokshe,KZ,/product/4870235430011-kz-yogurt-tureckiy-30-0...
4,Молочные продукты и яйца,Йогурт,Йогурт 1.5% Клубника-малина Nәtige т/п 450г,Nәtige,KZ,/product/4870003751300-kz-yogurt-0450kg-15-zhi...
...,...,...,...,...,...,...
5856,Табачные изделия,Сигареты,Сигареты с фильтром State Line 20шт,State Line,KZ,/product/48743914-kz-sigarety-jti-state-line-kor
5857,Табачные изделия,Сигареты,Сигареты с фильтром Winston Silver 20шт,Winston,NaN,/product/48741200-kz-sigarety-jti-winston-silv...
5858,Табачные изделия,Сигареты,СИГАРЕТЫ PHILIP MORRIS PARLIAMENT RESERVE КОР,Parliament,KZ,/product/48741514-kz-sigarety-philip-morris-pa...
5859,Табачные изделия,Сигареты,Сигареты с угольным фильтром Kent HDi Silver 20шт,Kent,RU,/product/46110312-kz-sigarety-vat-kent-hdi-sil...


# Some statistics...

In [113]:
df2 = df[['Category', 'Url of Product']].groupby(['Category']).count()
df2 = df2.rename(columns = {'Url of Product' : 'Count'})
df2.reset_index(inplace = True)
df2

,Category,Count
0,Алкоголь,771
1,Бакалея,402
2,Безалкогольные напитки,643
3,Детское питание,201
4,Кондитерские изделия,642
5,Молочные продукты и яйца,900
6,Мясные продукты,429
7,Овощи,208
8,Полуфабрикаты,271
9,Рыба,234


In [114]:
import plotly.express as px

fig = px.pie(df2, values='Count', names='Category', title='Products diversity')
fig.show()

In [108]:
fig = px.bar(df2, x='Category', y='Count', color="Category", color_discrete_sequence=px.colors.sequential.Plasma)
fig.show()

In [109]:
df3 = df[['Country', 'Url of Product']].groupby(['Country']).count()
df3 = df3.rename(columns = {'Url of Product' : 'Count'})
df3.reset_index(inplace = True)
df3

,Country,Count
0,AM,19
1,AR,1
2,AU,10
3,BE,3
4,BY,94
5,CH,23
6,CL,3
7,CN,49
8,CU,6
9,DE,48


In [110]:
fig = px.bar(df3, y='Count', x='Country', title='Countries and product count', color="Country")
fig.show()